# Wordle Solver

In [1]:
from wordle.utils import Wordle

%load_ext autoreload
%autoreload 2

### Play Wordle

In [2]:
# Initialize Wordle class
wordle = Wordle()

# Main function that returns a list of possible words
green_letters = ""
green_positions = []
yellow_letters = "taatlat"
yellow_positions = [1, 3, 2, 3, 4, 2, 5]
gray_letters = "irepyfgo"

# Find words in a list of possible words
words = wordle.find_words(
    green_letters,
    green_positions,
    yellow_letters,
    yellow_positions,
    gray_letters,
    answer_word_list=True,
)

# Choose a word to play
wordle.choose_word_to_play(word_threshold=2)

Number of possible words: 1
atlas 0.0


In [3]:
# Repetitive letters
letters_df = wordle.repetitive_letters()
print(letters_df.to_string(index=False))

Letters  Count
      A      2
      T      1
      L      1
      S      1


In [4]:
df = wordle.load_data()
df

,Names,Games_Won
0,Murilo,334
1,Barbara,145
2,Draw,326


In [5]:
# Update score
score_df = wordle.update_score()
score_df

,Names,Games_Won
0,Murilo,334
1,Barbara,145
2,Draw,327


In [6]:
# Print results
print(score_df.to_string(index=False))

  Names  Games_Won
 Murilo        334
Barbara        145
   Draw        327


#### Git commit message

In [7]:
# git commit message
commit_message = wordle.get_commit_message()
print(commit_message)

git commit -m "updated wordle 334-145-327"


#### Set custom score

In [8]:
# help(wordle.reset_score)

In [9]:
# help(wordle.set_score)

In [10]:
# # Set custom score
# df = wordle.set_score(m_score=311, b_score=132, draw_score=308)
# df

#### Reset score

In [11]:
## Reset score
# wordle.reset_score()